In [ ]:
import threading
import pygame
import time

# Function to run the environment updates in a separate thread
def run_env():
    r = 0
    done = False
    time.sleep(0.1) 
    while not done:
        observation, reward, done, info = env.step(env.action_space.sample())
        print(reward)
         # Sleep for 0.1 seconds between steps

# Your game environment and reset logic here
for i in range(5):
    env = StreetFighterEnv()
    state, game = env.reset()

    # Start the environment interactions in a new thread
    env_thread = threading.Thread(target=run_env)
    env_thread.start()

    # Main thread runs the game loop
    game.mainloop()

    # Wait for the environment thread to finish before quitting
    env_thread.join()

    # Quit pygame after the game loop ends
    pygame.quit()



In [5]:
from gym import Env
from gym import spaces
import Round

In [1]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import Round
import pygame
from pygame.locals import KEYDOWN
import menu
from game import Point
import csv
mapped_key={0: 1073741915,
            1: 1073741916,
            2: 1073741917,
            3: 1073741918,
            4: 1073741919,
            5: 1073741920,
            6: 1073741921}
def log_data_with_actions(observation, player1_action, player2_action):
    row = observation + [player1_action, player2_action]
    with open("data.csv", mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(row)

class StreetFighterEnv(gym.Env):
    def __init__(self):
        super(StreetFighterEnv, self).__init__()
        
        # Observation space: Positions and health normalized between 0 and 1
        # 6 values: x1, y1, health1, x2, y2, health2
        self.observation_space = spaces.Box(
            low=0.0,
            high=1.0,
            shape=(8,),  # 8 features: normalized positions,health and energy
            dtype=np.float32
        )
        
        # Action space: 7 discrete actions (4 movements + 2 attacks + 1 special)
        self.action_space = spaces.Discrete(8)
        
        # Game-specific attributes

        # pygame.init()
        # pygame.mixer.init()
        self.screen_width = 320
        self.screen_height = 240
        # self.screen = pygame.display.set_mode((320, 240), 0, 32)
        # self.player1 = Round.Player('Ken', 120, 100)
        # self.player2 = Round.Player('Ken', 120, 100, Player2=True,alt_color = True)
        # print('loading backgRound...')
        # self.backgRound = Round.Background('../res/BackgRound/Bckgrnd0.png')
        # print('creating game...')
        # game = Round.Game(self.screen,self.backgRound,self.player1,self.player2)
        

        
    def reset(self,seed=None):
        """Reset the environment to its initial state."""
        # Reset player states (example logic)
        pygame.init()
        pygame.mixer.init()
        self.screen = pygame.display.set_mode((640, 480), 0, 32)
        self.player1 = Round.Player('Ken', 120, 100)
        self.player2 = Round.Player('Ken', 120, 100, Player2=True,alt_color = True)
        print('loading backgRound...')
        self.backgRound = Round.Background('../res/BackgRound/Bckgrnd0.png')
        print('creating game...')
        self.game = Round.Game(self.screen,self.backgRound,self.player1,self.player2)
        Round.round_marker = (False, False)
        Round.end_round = False
        Round.ready_text = menu.Text('Ready',Point(128,120))
        Round.fight_text = menu.Text('Fight !!',Point(96,120))
        
        
        # Return the initial observation
        return self._get_observation(),{}
    #,self.game
    
    def step(self, action):
        """Execute one step in the environment."""
        # Apply the chosen action (implement your own logic here)
        #self._apply_action(self.player1, action)
        
        # Update the game state (you may have a game loop or logic to handle       
        # Get the current observation
        i1,ac2=self.game.mainloop(action)
        observation = self._get_observation()
        reward=0
        # Calculate the reward (implement your reward logic)
        #reward = self._calculate_reward()
        if action == 7:
            reward=1
        if i1 !=None:
            reward+=5
        
        # Determine if the game is done
        done = self._is_done()
        info={}
        truncated = False
        
        return observation, reward, done,truncated, info
    
    def _get_observation(self):
        """Generate the observation for the current state."""
        return np.array([
            self.player1.position.x / self.screen_width,
            self.player1.position.y / self.screen_height,
            self.player1.health.hp / 1000,  # Normalized health1
            self.player1.energy.energy / 96,
            self.player2.position.x / self.screen_width,
            self.player2.position.y / self.screen_height,
            self.player2.health.hp / 1000,   # Normalized health2
            self.player2.energy.energy / 96
        ], dtype=np.float32)
    
    def _apply_action(self, player, action):
        """Apply the chosen action to the player."""
        pygame.event.post(pygame.event.Event(KEYDOWN, key=mapped_key))
    
    # def _update_game_state(self):
    #     """Update the state of the game (e.g., handle collisions, time)."""
    #     pass  # Implement game logic
    
    def _calculate_reward(self):
        """Calculate the reward for the current state."""
        # Example: Reward based on damage dealt to the opponent
        damage_dealt = 1000 - self.player2.health.hp
        return damage_dealt
    
    def _is_done(self):
        """Check if the game is over."""
        # Example: Game ends if one player's health reaches 0
        return Round.end_round

In [ ]:
env=StreetFighterEnv()
env.reset()

In [ ]:
while True:
    env.step(env.action_space.sample())


In [ ]:
pygame.quit()
exit()

In [ ]:

import time
import sys
env=StreetFighterEnv()
for i in range(2):
    state,a = env.reset()
    r=0
    done=False
    while not done:
       obs, reward, done,truncated, info=env.step(env.action_space.sample())
time.sleep(0.2)
pygame.quit()

In [3]:
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import numpy as np

In [10]:
env=StreetFighterEnv()
#model = PPO.load("ppo_streetfighter", env=env)
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=1)
model.save("ppo_streetfighter")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
reading...../res/Char/Ken/KenHB.png
reading...../res/Char/Ken/KenHit.png
reading move list...
reading...../res/fireballsHit.png
reading...../res/Char/Ken/KenHB.png
reading...../res/Char/Ken/KenHit.png
reading move list...
reading...../res/fireballsHit.png
loading backgRound...
creating game...
-----------------------------
| time/              |      |
|    fps             | 29   |
|    iterations      | 1    |
|    time_elapsed    | 68   |
|    total_timesteps | 2048 |
-----------------------------


In [4]:
model = PPO.load("ppo_streetfighter")
obs,a = env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done,truncated, info = env.step(action)
    print(obs, reward, done, info)

In [1]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
import Round
import pygame
from pygame.locals import KEYDOWN
import menu
from game import Point
mapped_key={0: 1073741915,
            1: 1073741916,
            2: 1073741917,
            3: 1073741918,
            4: 1073741919,
            5: 1073741920,
            6: 1073741921}

def log_data_with_actions(observation, player1_action, player2_action, file_path="data.csv"):
    row = observation + [player1_action, player2_action]
    with open(file_path, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(row)

class StreetFighterEnvInfiTrain(gym.Env):
    def __init__(self):
        super(StreetFighterEnvInfiTrain, self).__init__()
        
        # Observation space: Positions and health normalized between 0 and 1
        # 6 values: x1, y1, health1, x2, y2, health2
        self.observation_space = spaces.Box(
            low=0.0,
            high=1.0,
            shape=(8,),  # 8 features: normalized positions,health and energy
            dtype=np.float32
        )
        
        # Action space: 7 discrete actions (4 movements + 2 attacks + 1 special)
        self.action_space = spaces.Discrete(8)
        
        # Game-specific attributes

        # pygame.init()
        # pygame.mixer.init()
        self.screen_width = 320
        self.screen_height = 240
        # self.screen = pygame.display.set_mode((320, 240), 0, 32)
        # self.player1 = Round.Player('Ken', 120, 100)
        # self.player2 = Round.Player('Ken', 120, 100, Player2=True,alt_color = True)
        # print('loading backgRound...')
        # self.backgRound = Round.Background('../res/BackgRound/Bckgrnd0.png')
        # print('creating game...')
        # game = Round.Game(self.screen,self.backgRound,self.player1,self.player2)
        

        
    def reset(self,seed=None):
        """Reset the environment to its initial state."""
        # Reset player states (example logic)
        pygame.init()
        pygame.mixer.init()
        self.screen = pygame.display.set_mode((640, 480), 0, 32)
        self.player1 = Round.Player('Ken', 120, 100)
        self.player2 = Round.Player('Ken', 120, 100, Player2=True,alt_color = True)
        print('loading backgRound...')
        self.backgRound = Round.Background('../res/BackgRound/Bckgrnd0.png')
        print('creating game...')
        self.game = Round.Game(self.screen,self.backgRound,self.player1,self.player2)
        Round.round_marker = (False, False)
        Round.end_round = False
        Round.ready_text = menu.Text('Ready',Point(128,120))
        Round.fight_text = menu.Text('Fight !!',Point(96,120))
        self.model = PPO.load("ppo_streetfighter1")
        self.fire=0
        
        
        # Return the initial observation
        return self._get_observation(),{}
    #,self.game
    
    def step(self, action):
        """Execute one step in the environment."""
        # Apply the chosen action (implement your own logic here)
        #self._apply_action(self.player1, action)
        
        # Update the game state (you may have a game loop or logic to handle       
        # Get the current observation
        
        observation = self._get_observation()
        reward=0
        action2, _states = self.model.predict(observation)
        i1=self.game.mainloop(action,action2)
        # Calculate the reward (implement your reward logic)
        #reward = self._calculate_reward()
        # if action == 7:
        #     reward+=1
        if i1 !=None:
            reward+=5
        self.fire+=reward
        # Determine if the game is done
        done = self._is_done()
        info={}
        truncated = False
        if done:
            print(self.fire)
        log_data_with_actions(observation,action,action2)
        return observation, reward, done,truncated, info
    
    def _get_observation(self):
        """Generate the observation for the current state."""
        return np.array([
            self.player1.position.x / self.screen_width,
            self.player1.position.y / self.screen_height,
            self.player1.health.hp / 1000,  # Normalized health1
            self.player1.energy.energy / 96,
            self.player2.position.x / self.screen_width,
            self.player2.position.y / self.screen_height,
            self.player2.health.hp / 1000,   # Normalized health2
            self.player2.energy.energy / 96
        ], dtype=np.float32)
    
    def _apply_action(self, player, action):
        """Apply the chosen action to the player."""
        pygame.event.post(pygame.event.Event(KEYDOWN, key=mapped_key))
    
    # def _update_game_state(self):
    #     """Update the state of the game (e.g., handle collisions, time)."""
    #     pass  # Implement game logic
    
    def _calculate_reward(self):
        """Calculate the reward for the current state."""
        # Example: Reward based on damage dealt to the opponent
        damage_dealt = 1000 - self.player2.health.hp
        return damage_dealt
    
    def _is_done(self):
        """Check if the game is over."""
        # Example: Game ends if one player's health reaches 0
        return Round.end_round

In [4]:

import time
import sys
env=StreetFighterEnvInfiTrain()
for i in range(2):
    state,a = env.reset()
    r=0
    done=False
    while not done:
       obs, reward, done,truncated, info=env.step(env.action_space.sample())
time.sleep(0.2)
pygame.quit()

In [13]:
env=StreetFighterEnvInfiTrain()
model = PPO.load("ppo_streetfighter", env=env)
model.learn(total_timesteps=10)
model.save("ppo_streetfighter")

In [5]:


env=StreetFighterEnvInfiTrain()
model = PPO.load("ppo_streetfighter1", env=env)
model.learn(total_timesteps=10000)
model.save("ppo_streetfighter2")

In [ ]:
exit()

: 